In [18]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import pickle, gzip
import gc
from sklearn.model_selection import train_test_split

In [19]:
train_series = pd.read_csv('Data/training_set.csv')
metadata_train = pd.read_csv('Data/training_set_metadata.csv')

simple_features = train_series.groupby(
    ['object_id', 'passband'])['flux', 'flux_err'].agg(
    ['mean', 'median', 'max', 'min', 'std']).unstack('passband')


#construct time series using binned observations:
ts_mod = train_series[['object_id', 'mjd', 'passband', 'flux', 'flux_err']].copy()
#bin by 5 days, reducing the size of data but still giving a time series
ts_mod['mjd_d5'] = (ts_mod['mjd'] / 5).astype(int)
ts_mod = ts_mod.groupby(['object_id', 'mjd_d5', 'passband'])['flux', 'flux_err'].mean().reset_index()

#pivotting
ts_piv = pd.pivot_table(ts_mod, 
                        index='object_id', 
                        columns=['mjd_d5', 'passband'], 
                        values=['flux', 'flux_err'],
                        dropna=False)

gc.enable()
ts_piv.head()

flux                                       ...  flux_err            \
mjd_d5    11916                     11917             ...     12133             
passband      0   1   2   3   4   5     0   1   2   3 ...         2         3   
object_id                                             ...                       
615         NaN NaN NaN NaN NaN NaN   NaN NaN NaN NaN ...       NaN       NaN   
713         NaN NaN NaN NaN NaN NaN   NaN NaN NaN NaN ...  0.798698  1.242993   
730         NaN NaN NaN NaN NaN NaN   NaN NaN NaN NaN ...       NaN       NaN   
745         NaN NaN NaN NaN NaN NaN   NaN NaN NaN NaN ...       NaN       NaN   
1124        NaN NaN NaN NaN NaN NaN   NaN NaN NaN NaN ...       NaN       NaN   

                                                             
mjd_d5                            12134                      
passband          4         5         0   1   2   3   4   5  
object_id                                                    
615             NaN       NaN       NaN NaN NaN NaN NaN NaN  
713        1.853826  4.562611  1.913871 NaN NaN NaN NaN NaN  
730             NaN       NaN       NaN NaN NaN NaN NaN NaN  
745             NaN       NaN       NaN NaN NaN NaN NaN NaN  
1124            NaN       NaN       NaN NaN NaN NaN NaN NaN  

[5 rows x 2532 columns]

In [20]:
del metadata_train['ra'],metadata_train['decl'],metadata_train['gal_l'], metadata_train['gal_b'],metadata_train['hostgal_photoz'],metadata_train['hostgal_photoz_err'], metadata_train['distmod'], metadata_train['mwebv']

#Bin into ddf and non-ddf training
ddf = metadata_train[(metadata_train['ddf'] == 1)]
del ddf['ddf']

ddf_far_away= (ddf[(ddf['hostgal_specz'] > 0)])
ddf_far_away.set_index('object_id', inplace=True)

ddf_nearby= ddf[(ddf['hostgal_specz'] <=0)]
ddf_nearby.set_index('object_id', inplace=True)

non_ddf = metadata_train[(metadata_train['ddf'] == 0)]
del non_ddf['ddf']

non_ddf_far_away= non_ddf[(non_ddf['hostgal_specz'] >0)]
non_ddf_far_away.set_index('object_id', inplace=True)

non_ddf_nearby= non_ddf[(non_ddf['hostgal_specz'] <=0 )]
non_ddf_nearby.set_index('object_id', inplace=True)

del ddf, non_ddf, ddf_far_away['hostgal_specz'], non_ddf_far_away['hostgal_specz'], ddf_nearby['hostgal_specz'], non_ddf_nearby['hostgal_specz']
gc.collect()


265

In [25]:
#Split into validation and training. 

ddf_far_away_train, ddf_far_away_validation = train_test_split(ddf_far_away, test_size=0.1)
ddf_nearby_train, ddf_nearby_validation = train_test_split(ddf_nearby, test_size=0.1)

non_ddf_far_away_train, non_ddf_far_away_validation = train_test_split(non_ddf_far_away, test_size=0.1)
non_ddf_nearby_train, non_ddf_nearby_validation = train_test_split(non_ddf_nearby, test_size=0.1)





In [36]:
import numpy as np
import torch
import torch.nn as nn

In [59]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

rnn = RNN(6, 32, 13)

In [60]:
learning_rate = 0.005
criterion = nn.NLLLoss()

def train(y_train, x_train):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(x_train.size()[0]):
        output, hidden = rnn(x_train[i], hidden)

    loss = criterion(output, y_train)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        gd = p.grad.data
        gd[gd != gd] = 0
        p.data.add_(-learning_rate, gd)

    return output, loss.item()

In [70]:
classes = tuple(metadata_train.target.unique())
print(classes)

(92, 88, 42, 90, 65, 16, 67, 95, 62, 15, 52, 6, 64)


In [67]:
test_x = torch.tensor(ts_piv.loc[713].values.reshape(-1, 1, 6), dtype = torch.float32)
test_y = torch.tensor([classes.index(ddf_far_away.loc[713].values)])
y = ddf_far_away.loc[713].values[0]

In [68]:
test_x[test_x != test_x] = 0

In [69]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return classes[category_i], category_i


(88, 1)


In [65]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(test_y, test_x)
    current_loss += loss

    if iter % print_every == 0:
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, correct))
    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0
        
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

KeyboardInterrupt: 

In [72]:
torch.exp(output)

tensor([[4.7035e-08, 1.0000e+00, 1.0862e-08, 3.5450e-08, 2.9733e-07, 9.1874e-08,
         6.2914e-08, 1.5906e-07, 1.8016e-07, 4.6697e-08, 3.0166e-08, 9.4686e-08,
         5.6874e-10]], grad_fn=<ExpBackward>)